In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc

import time

import missingno

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from statsmodels.stats.outliers_influence import variance_inflation_factor

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차 검증
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 모델의 최적의 하이퍼 파라미터를 찾기 위한 도구
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import RandomizedSearchCV

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

# 학습 모델 저장을 위한 라이브러리
import pickle

In [2]:
df1 = pd.read_csv('2024년_수비_스탯_연봉_데이터(포지션)최종.csv')
df1

,선수,팀,WAR,G,oWAR,dWAR,PA,ePA,AB,R,...,OBP,SLG,OPS,R/ePA,wRC+,pid,연도,기존pid,연봉(만원),포지션
0,노시환,한화 이글스,2.41,136,2.50,-0.08,601,597,526,88,...,0.356,0.454,0.810,0.145,106.5,141259,2024,14125,35000.0,3루수
1,페라자,한화 이글스,2.13,122,2.27,-0.14,522,518,455,75,...,0.364,0.486,0.850,0.158,116.7,161219,2024,16121,104000.0,좌익수
2,최재훈,한화 이글스,1.82,116,1.69,0.13,348,340,280,34,...,0.371,0.336,0.707,0.131,96.6,101709,2024,10170,60000.0,포수
3,황영묵,한화 이글스,1.39,123,1.38,0.01,389,386,349,52,...,0.365,0.372,0.737,0.129,95.1,161109,2024,16110,3000.0,2루수
4,안치홍,한화 이글스,1.33,128,1.65,-0.31,533,531,473,64,...,0.370,0.427,0.797,0.146,107.9,100149,2024,10014,50000.0,1루수
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,고영우,키움 히어로즈,-0.42,94,0.01,-0.43,290,286,254,21,...,0.329,0.319,0.648,0.094,73.5,161287,2024,16128,3000.0,3루수
293,이원석,키움 히어로즈,-0.53,39,-0.33,-0.20,109,109,91,5,...,0.330,0.264,0.594,0.085,66.4,101807,2024,10180,40000.0,1루수
294,이승원,키움 히어로즈,-0.74,20,-0.53,-0.20,41,39,37,3,...,0.154,0.135,0.289,-0.058,-43.9,154767,2024,15476,3100.0,유격수
295,임지열,키움 히어로즈,-0.87,22,-0.59,-0.28,57,57,49,4,...,0.211,0.163,0.374,-0.011,-8.3,112257,2024,11225,7200.0,우익수


In [3]:
df2 = pd.read_csv('투수_스탯_연봉_파생변수(금액).csv')
df2

,선수,G,GS,GR,GF,CG,SHO,W,L,S,...,WHIP,WAR,팀,pid,연도,연봉(만원),신인여부,국적,선발투수,연봉등급
0,페냐,32,32,0,0,0,0,11,11,0,...,1.17,4.28,한화 이글스,15146,2024,65000,False,외국인,1,A
1,문동주,23,23,0,0,0,0,8,8,0,...,1.31,3.08,한화 이글스,15013,2024,10000,False,내국인,1,B
2,산체스,24,24,0,0,0,0,7,8,0,...,1.30,2.59,한화 이글스,15643,2024,50000,False,외국인,1,A
3,주현상,55,0,55,9,0,0,2,2,0,...,0.84,2.44,한화 이글스,11415,2024,11000,False,내국인,0,B
4,이태양,50,12,38,6,0,0,3,3,0,...,1.21,1.78,한화 이글스,10609,2024,50000,False,내국인,0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,박범준,5,0,5,3,0,0,0,1,0,...,2.06,-0.19,키움 히어로즈,16133,2025,7100,True,내국인,0,C
422,전준표,18,5,13,4,0,0,2,4,0,...,1.99,-0.35,키움 히어로즈,16122,2025,3200,False,내국인,0,D
423,오석주,17,0,17,7,0,0,1,1,0,...,2.29,-0.50,키움 히어로즈,12860,2025,3900,False,내국인,0,D
424,조영건,25,3,22,3,0,0,2,1,0,...,1.91,-0.55,키움 히어로즈,14132,2025,3800,False,내국인,0,D


In [4]:
# 연도가 2023 인 컬럼
pitcher_2024 = df2[df2['연도'] == 2024]
pitcher_2024

,선수,G,GS,GR,GF,CG,SHO,W,L,S,...,WHIP,WAR,팀,pid,연도,연봉(만원),신인여부,국적,선발투수,연봉등급
0,페냐,32,32,0,0,0,0,11,11,0,...,1.17,4.28,한화 이글스,15146,2024,65000,False,외국인,1,A
1,문동주,23,23,0,0,0,0,8,8,0,...,1.31,3.08,한화 이글스,15013,2024,10000,False,내국인,1,B
2,산체스,24,24,0,0,0,0,7,8,0,...,1.30,2.59,한화 이글스,15643,2024,50000,False,외국인,1,A
3,주현상,55,0,55,9,0,0,2,2,0,...,0.84,2.44,한화 이글스,11415,2024,11000,False,내국인,0,B
4,이태양,50,12,38,6,0,0,3,3,0,...,1.21,1.78,한화 이글스,10609,2024,50000,False,내국인,0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,이종민,11,2,9,4,0,0,0,1,0,...,2.05,-0.29,키움 히어로즈,14576,2024,3200,False,내국인,0,D
202,김동규,3,1,2,0,0,0,0,1,0,...,3.75,-0.47,키움 히어로즈,15485,2024,3100,False,내국인,0,D
203,김동혁,35,2,33,11,0,0,1,7,0,...,1.75,-0.51,키움 히어로즈,14674,2024,6000,False,내국인,0,C
204,하영민,57,0,57,7,0,0,3,1,0,...,1.76,-0.70,키움 히어로즈,11222,2024,8000,False,내국인,0,C


In [5]:
pitcher_2024= pitcher_2024.rename(columns={
   
    'pid':'기존pid'
})

In [6]:
# 1. 타자 데이터프레임: 포지션구분 컬럼만 추가
df1['포지션구분'] = '타자'

# 2. 투수 데이터프레임: 포지션구분 컬럼만 추가
pitcher_2024['포지션구분'] = '투수'

# 3. 컬럼이 다를 수 있으므로 outer join 방식으로 병합
df_combined = pd.concat([df1, pitcher_2024], ignore_index=True, sort=False)

In [7]:
df_combined

,선수,팀,WAR,G,oWAR,dWAR,PA,ePA,AB,R,...,TBF,BK,WP,ERA,FIP,WHIP,신인여부,국적,선발투수,연봉등급
0,노시환,한화 이글스,2.41,136,2.50,-0.08,601.0,597.0,526.0,88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,페라자,한화 이글스,2.13,122,2.27,-0.14,522.0,518.0,455.0,75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,최재훈,한화 이글스,1.82,116,1.69,0.13,348.0,340.0,280.0,34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,황영묵,한화 이글스,1.39,123,1.38,0.01,389.0,386.0,349.0,52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,안치홍,한화 이글스,1.33,128,1.65,-0.31,533.0,531.0,473.0,64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,이종민,키움 히어로즈,-0.29,11,NaN,NaN,NaN,NaN,NaN,13,...,72.0,0.0,0.0,7.24,3.61,2.05,False,내국인,0.0,D
499,김동규,키움 히어로즈,-0.47,3,NaN,NaN,NaN,NaN,NaN,10,...,25.0,0.0,4.0,22.50,13.95,3.75,False,내국인,0.0,D
500,김동혁,키움 히어로즈,-0.51,35,NaN,NaN,NaN,NaN,NaN,34,...,192.0,2.0,0.0,7.32,4.52,1.75,False,내국인,0.0,C
501,하영민,키움 히어로즈,-0.70,57,NaN,NaN,NaN,NaN,NaN,36,...,247.0,0.0,6.0,4.64,3.86,1.76,False,내국인,0.0,C


In [8]:
df1.columns

Index(['선수', '팀', 'WAR', 'G', 'oWAR', 'dWAR', 'PA', 'ePA', 'AB', 'R', 'H',
       '2B', '3B', 'HR', 'TB', 'RBI', 'SB', 'CS', 'BB', 'HP', 'IB', 'SO',
       'GDP', 'SH', 'SF', 'AVG', 'OBP', 'SLG', 'OPS', 'R/ePA', 'wRC+', 'pid',
       '연도', '기존pid', '연봉(만원)', '포지션', '포지션구분'],
      dtype='object')

In [9]:
pitcher_2024.columns

Index(['선수', 'G', 'GS', 'GR', 'GF', 'CG', 'SHO', 'W', 'L', 'S', 'HD', 'IP',
       'ER', 'R', 'TBF', 'H', '2B', '3B', 'HR', 'BB', 'HP', 'SO', 'BK', 'WP',
       'ERA', 'FIP', 'WHIP', 'WAR', '팀', '기존pid', '연도', '연봉(만원)', '신인여부', '국적',
       '선발투수', '연봉등급', '포지션구분'],
      dtype='object')

In [10]:
pitcher_2024['포지션'] = '투수'

In [11]:
pitcher_2024

,선수,G,GS,GR,GF,CG,SHO,W,L,S,...,팀,기존pid,연도,연봉(만원),신인여부,국적,선발투수,연봉등급,포지션구분,포지션
0,페냐,32,32,0,0,0,0,11,11,0,...,한화 이글스,15146,2024,65000,False,외국인,1,A,투수,투수
1,문동주,23,23,0,0,0,0,8,8,0,...,한화 이글스,15013,2024,10000,False,내국인,1,B,투수,투수
2,산체스,24,24,0,0,0,0,7,8,0,...,한화 이글스,15643,2024,50000,False,외국인,1,A,투수,투수
3,주현상,55,0,55,9,0,0,2,2,0,...,한화 이글스,11415,2024,11000,False,내국인,0,B,투수,투수
4,이태양,50,12,38,6,0,0,3,3,0,...,한화 이글스,10609,2024,50000,False,내국인,0,A,투수,투수
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,이종민,11,2,9,4,0,0,0,1,0,...,키움 히어로즈,14576,2024,3200,False,내국인,0,D,투수,투수
202,김동규,3,1,2,0,0,0,0,1,0,...,키움 히어로즈,15485,2024,3100,False,내국인,0,D,투수,투수
203,김동혁,35,2,33,11,0,0,1,7,0,...,키움 히어로즈,14674,2024,6000,False,내국인,0,C,투수,투수
204,하영민,57,0,57,7,0,0,3,1,0,...,키움 히어로즈,11222,2024,8000,False,내국인,0,C,투수,투수


In [12]:
merged_2024 = pd.concat([pitcher_2024.assign(포지션구분='투수'),
                           df1.assign(포지션구분='타자')],
                          ignore_index=True)
merged_2024

,선수,G,GS,GR,GF,CG,SHO,W,L,S,...,GDP,SH,SF,AVG,OBP,SLG,OPS,R/ePA,wRC+,pid
0,페냐,32,32.0,0.0,0.0,0.0,0.0,11.0,11.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,문동주,23,23.0,0.0,0.0,0.0,0.0,8.0,8.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,산체스,24,24.0,0.0,0.0,0.0,0.0,7.0,8.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,주현상,55,0.0,55.0,9.0,0.0,0.0,2.0,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,이태양,50,12.0,38.0,6.0,0.0,0.0,3.0,3.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,고영우,94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,4.0,3.0,0.256,0.329,0.319,0.648,0.094,73.5,161287.0
499,이원석,39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,0.0,2.0,0.220,0.330,0.264,0.594,0.085,66.4,101807.0
500,이승원,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,0.0,0.108,0.154,0.135,0.289,-0.058,-43.9,154767.0
501,임지열,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,0.0,1.0,0.102,0.211,0.163,0.374,-0.011,-8.3,112257.0


In [13]:



# # 필요한 컬럼만 추출
# df1 = df1[['선수', '팀', 'WAR', '연도', '포지션', '연봉(만원)','pid']]
# pitcher_2024 = pitcher_2024[['선수', '팀', 'WAR', '연도', '포지션', '연봉(만원)','pid']]

# # 포지션구분 컬럼 추가
# df1['포지션구분'] = '타자'
# pitcher_2024['포지션구분'] = '투수'

# # 열 순서 맞추기
# common_columns = ['선수', '팀', '연도', '포지션', '포지션구분', 'WAR', '연봉(만원)']
# df1 = df1[common_columns]
# pitcher_2024 = pitcher_2024[common_columns]

# # 병합
# merged_df = pd.concat([df1, pitcher_2024], ignore_index=True)


In [14]:
# CSV 파일 불러오기
batter_df = pd.read_csv("2023년_수비_스탯_연봉_데이터(포지션)최종.csv")
pitcher_df = pd.read_csv("2023년투수최종.csv")


In [15]:
pitcher_df= pitcher_df.rename(columns={
   
    'pid':'기존pid'
})

In [16]:
# 병합
merged_2023 = pd.concat([batter_df, pitcher_df], ignore_index=True)
merged_2023

,선수,팀,WAR,G,oWAR,dWAR,PA,ePA,AB,R,...,HD,IP,ER,TBF,BK,WP,ERA,FIP,WHIP,연봉등급
0,노시환,한화 이글스,6.53,131,6.17,0.35,595.0,590.0,514.0,85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,최재훈,한화 이글스,3.23,125,3.03,0.20,417.0,407.0,327.0,23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,이진영,한화 이글스,2.44,121,2.06,0.38,422.0,418.0,358.0,57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,채은성,한화 이글스,1.79,137,1.95,-0.16,596.0,593.0,521.0,71,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,이도윤,한화 이글스,1.64,106,0.91,0.73,346.0,331.0,309.0,36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,김동규,키움 히어로즈,-0.47,3,NaN,NaN,NaN,NaN,NaN,10,...,0.0,4.0,10.0,25.0,0.0,4.0,22.50,13.95,3.75,D
541,김동혁,키움 히어로즈,-0.51,35,NaN,NaN,NaN,NaN,NaN,34,...,6.0,39.1,32.0,192.0,2.0,0.0,7.32,4.52,1.75,C
542,하영민,키움 히어로즈,-0.70,57,NaN,NaN,NaN,NaN,NaN,36,...,5.0,52.1,27.0,247.0,0.0,6.0,4.64,3.86,1.76,C
543,양현,키움 히어로즈,-0.74,54,NaN,NaN,NaN,NaN,NaN,38,...,8.0,57.0,32.0,262.0,1.0,0.0,5.05,5.28,1.82,C


In [17]:
# 두 데이터를 그대로 이어붙이기 (컬럼 동일하므로 concat 사용)
df_all = pd.concat([merged_2023, merged_2024], ignore_index=True)
df_all

,선수,팀,WAR,G,oWAR,dWAR,PA,ePA,AB,R,...,BK,WP,ERA,FIP,WHIP,연봉등급,신인여부,국적,선발투수,포지션구분
0,노시환,한화 이글스,6.53,131,6.17,0.35,595.0,590.0,514.0,85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,최재훈,한화 이글스,3.23,125,3.03,0.20,417.0,407.0,327.0,23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,이진영,한화 이글스,2.44,121,2.06,0.38,422.0,418.0,358.0,57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,채은성,한화 이글스,1.79,137,1.95,-0.16,596.0,593.0,521.0,71,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,이도윤,한화 이글스,1.64,106,0.91,0.73,346.0,331.0,309.0,36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1043,고영우,키움 히어로즈,-0.42,94,0.01,-0.43,290.0,286.0,254.0,21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,타자
1044,이원석,키움 히어로즈,-0.53,39,-0.33,-0.20,109.0,109.0,91.0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,타자
1045,이승원,키움 히어로즈,-0.74,20,-0.53,-0.20,41.0,39.0,37.0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,타자
1046,임지열,키움 히어로즈,-0.87,22,-0.59,-0.28,57.0,57.0,49.0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,타자


In [18]:
df_2122 = pd.read_csv('2122년_투수타자_데이터.csv')
df_2122

,선수,Team,Sort▼,G,GS,GR,GF,CG,SHO,W,...,CS RAA,Blk RAA,Frm RAA,종합 RAA,/144,POSAdj,RAAwithPOS,WAAwoPOS,pid_new,WAR▼
0,킹험,투수,4.59,25,25,0.0,0.0,0.0,0.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,김민우,투수,4.40,29,28,1.0,0.0,0.0,0.0,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,카펜터,투수,3.25,31,30,1.0,0.0,0.0,0.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,강재민,투수,2.22,58,0,58.0,18.0,0.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,윤대경,투수,1.71,43,9,34.0,2.0,0.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363,임종찬,우익수,NaN,20,17,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.39,2.808,-0.513,-0.123,0.040,147069.0,NaN
1364,전병우,1루수,NaN,116,43,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,3.89,4.829,-3.658,0.232,0.401,114227.0,NaN
1365,권광민,좌익수,NaN,29,20,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,-1.14,-5.661,-0.450,-1.590,-0.118,136769.0,NaN
1366,노수광,좌익수,NaN,104,83,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,2.34,3.240,-2.620,-0.280,0.241,112689.0,NaN


In [19]:
df_2122= df_2122.rename(columns={
    'Team': '포지션',
    'pid':'기존pid',
    'pid_new':'pid'
})

In [20]:
df_final = pd.concat([df_2122, df_all], ignore_index=True)
df_final

,선수,포지션,Sort▼,G,GS,GR,GF,CG,SHO,W,...,AVG,OBP,SLG,OPS,R/ePA,wRC+,연봉등급,신인여부,국적,선발투수
0,킹험,투수,4.59,25,25.0,0.0,0.0,0.0,0.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,김민우,투수,4.40,29,28.0,1.0,0.0,0.0,0.0,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,카펜터,투수,3.25,31,30.0,1.0,0.0,0.0,0.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,강재민,투수,2.22,58,0.0,58.0,18.0,0.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,윤대경,투수,1.71,43,9.0,34.0,2.0,0.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2411,고영우,3루수,NaN,94,NaN,NaN,NaN,NaN,NaN,NaN,...,0.256,0.329,0.319,0.648,0.094,73.5,NaN,NaN,NaN,NaN
2412,이원석,1루수,NaN,39,NaN,NaN,NaN,NaN,NaN,NaN,...,0.220,0.330,0.264,0.594,0.085,66.4,NaN,NaN,NaN,NaN
2413,이승원,유격수,NaN,20,NaN,NaN,NaN,NaN,NaN,NaN,...,0.108,0.154,0.135,0.289,-0.058,-43.9,NaN,NaN,NaN,NaN
2414,임지열,우익수,NaN,22,NaN,NaN,NaN,NaN,NaN,NaN,...,0.102,0.211,0.163,0.374,-0.011,-8.3,NaN,NaN,NaN,NaN


In [21]:
df_final.to_csv("2124_타자투수_스텟_통합.csv", index=False)


In [22]:
df_final.columns

Index(['선수', '포지션', 'Sort▼', 'G', 'GS', 'GR', 'GF', 'CG', 'SHO', 'W', 'L', 'S',
       'HD', 'IP', 'ER', 'R', 'rRA', 'TBF', 'H', '2B', '3B', 'HR', 'BB', 'HP',
       'IB', 'SO', 'ROE', 'BK', 'WP', 'ERA', 'RA9', 'rRA9', 'rRA9pf', 'FIP',
       'WHIP', 'WAR', '팀', '기존pid', '연봉(만원)', '연도', '포지션구분', 'WAAwithPOS',
       'TC', 'PO', 'Ass', 'E', 'F%', 'RF9', 'Range RAA', 'Arm RAA', 'Err RAA',
       'DP RAA', 'Bt RAA', 'CS RAA', 'Blk RAA', 'Frm RAA', '종합 RAA', '/144',
       'POSAdj', 'RAAwithPOS', 'WAAwoPOS', 'pid', 'WAR▼', 'oWAR', 'dWAR', 'PA',
       'ePA', 'AB', 'TB', 'RBI', 'SB', 'CS', 'GDP', 'SH', 'SF', 'AVG', 'OBP',
       'SLG', 'OPS', 'R/ePA', 'wRC+', '연봉등급', '신인여부', '국적', '선발투수'],
      dtype='object')

In [24]:
df_final[df_final['포지션'].isnull()]


,선수,포지션,Sort▼,G,GS,GR,GF,CG,SHO,W,...,AVG,OBP,SLG,OPS,R/ePA,wRC+,연봉등급,신인여부,국적,선발투수
1443,박형준,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000,0.000,0.000,0.000,-0.167,-140.9,NaN,NaN,NaN,NaN
1560,류효승,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000,0.250,0.000,0.250,0.010,8.5,NaN,NaN,NaN,NaN
2195,백두산,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000,0.000,0.000,0.000,-0.190,-127.9,NaN,NaN,NaN,NaN
2289,김동엽,NaN,NaN,8,NaN,NaN,NaN,NaN,NaN,NaN,...,0.111,0.190,0.167,0.357,-0.031,-21.5,NaN,NaN,NaN,NaN
2307,현원회,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000,0.000,0.000,0.000,-0.190,-130.7,NaN,NaN,NaN,NaN
2309,류효승,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000,0.000,0.000,0.000,-0.190,-146.7,NaN,NaN,NaN,NaN
2312,신범수,NaN,NaN,11,NaN,NaN,NaN,NaN,NaN,NaN,...,0.222,0.276,0.333,0.609,0.074,51.8,NaN,NaN,NaN,NaN
2399,김수환,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000,0.000,0.000,0.000,-0.190,-154.5,NaN,NaN,NaN,NaN
2410,김웅빈,NaN,NaN,12,NaN,NaN,NaN,NaN,NaN,NaN,...,0.125,0.243,0.125,0.368,0.002,1.5,NaN,NaN,NaN,NaN


In [26]:
df_final = df_final.dropna(subset=['포지션'])


In [28]:
df_final

,선수,포지션,Sort▼,G,GS,GR,GF,CG,SHO,W,...,AVG,OBP,SLG,OPS,R/ePA,wRC+,연봉등급,신인여부,국적,선발투수
0,킹험,투수,4.59,25,25.0,0.0,0.0,0.0,0.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,김민우,투수,4.40,29,28.0,1.0,0.0,0.0,0.0,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,카펜터,투수,3.25,31,30.0,1.0,0.0,0.0,0.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,강재민,투수,2.22,58,0.0,58.0,18.0,0.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,윤대경,투수,1.71,43,9.0,34.0,2.0,0.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2411,고영우,3루수,NaN,94,NaN,NaN,NaN,NaN,NaN,NaN,...,0.256,0.329,0.319,0.648,0.094,73.5,NaN,NaN,NaN,NaN
2412,이원석,1루수,NaN,39,NaN,NaN,NaN,NaN,NaN,NaN,...,0.220,0.330,0.264,0.594,0.085,66.4,NaN,NaN,NaN,NaN
2413,이승원,유격수,NaN,20,NaN,NaN,NaN,NaN,NaN,NaN,...,0.108,0.154,0.135,0.289,-0.058,-43.9,NaN,NaN,NaN,NaN
2414,임지열,우익수,NaN,22,NaN,NaN,NaN,NaN,NaN,NaN,...,0.102,0.211,0.163,0.374,-0.011,-8.3,NaN,NaN,NaN,NaN


In [30]:
df_final.to_csv("2124_타자투수_스텟_통합.csv", index=False)


In [23]:
10/0

ZeroDivisionError: division by zero

In [ ]:
df_2023 = pd.read_csv('2023_타자_투수_통합.csv')
df_2023

In [ ]:
df_2024 = pd.read_csv('2024_타자_투수_통합.csv')
df_2024

In [ ]:
# 두 데이터를 그대로 이어붙이기 (컬럼 동일하므로 concat 사용)
df_all = pd.concat([df_2023, df_2024], ignore_index=True)

In [ ]:
df_all

In [ ]:
df_all.to_csv("2324_타자_투수_통합.csv", index=False)


In [ ]:
bat2025 = pd.read_csv('2025년_수비_스탯_연봉_데이터.csv')
bat2025